In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
from scipy import sparse, linalg
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange

np.random.seed(0)

# Define macros

# Physical constants
K = 9 # number of fermionic modes
J = 4 # ~"energy scale"
Q = 4 # order of coupling
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions

N_SAMPLES = 100 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel

## 1. Define fermionic modes

In [2]:
cr = sparse.csr_array(np.array([[0,1],[0,0]]))
an = sparse.csr_array(np.array([[0,0],[1,0]]))
id = sparse.csr_array(np.identity(2))
id2 = sparse.csr_array(np.array([[-1,0],[0,1]]))

def c(n):
    factors = [id for i in range(n-1)]+[cr]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = sparse.kron(out,factors[i])
    return out

def cd(n):
    factors = [id for i in range(n-1)]+[an]+[id2 for i in range(K-n)]
    out = factors[0]
    for i in range(1, K):
        out = sparse.kron(out,factors[i])
    return out

### 1.a. Check that fermionic modes satisfy algebra

$\{c_i, c_j\}=\{c_i^\dagger, c_j^\dagger\}=0$,     $\{c_i,c_j^\dagger\}=\delta_{ij}$

In [3]:
def anticommutator(a,b):
    return a@b+b@a
    
c_algebra_satisfied = True
tic = time.time()
for i in range(1,K+1):
    c_i = c(i)
    cd_i = cd(i)

    for j in range(1,K+1):
        c_j = c(j)
        cd_j = cd(j)

        ac_ci_cj = anticommutator(c_i, c_j).toarray()
        if not np.allclose(ac_ci_cj, np.zeros((N_DIM, N_DIM))):
            c_algebra_satisfied = False

        ac_cdi_cdj = anticommutator(cd_i, cd_j).toarray()
        if not np.allclose(ac_cdi_cdj, np.zeros((N_DIM, N_DIM))):
            c_algebra_satisfied = False

        ac_ci_cdj = anticommutator(c_i, cd_j).toarray()
        if i==j:
            if not np.allclose(ac_ci_cdj, np.identity(N_DIM)):
                c_algebra_satisfied = False
        else:
            if not np.allclose(ac_ci_cdj, np.zeros((N_DIM, N_DIM))):
                c_algebra_satisfied = False

toc = time.time()
duration = toc - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"Fermionic mode algebra satisfied: {c_algebra_satisfied}")

Duration: 0.0 minutes, 4.700317621231079 seconds
Fermionic mode algebra satisfied: True


scipy.sparse version is slower at $K=7$, about as fast at $K=8$, slightly faster at $K=9$, and undisputably faster at $K=11$

## 2. Define fermions

Note the sparse_array optimization

In [4]:
psi = [None for i in range(N)] 
for i in range(1,K+1):
    psi[2*(i-1)] = sparse.csr_matrix((c(i)+cd(i))/np.sqrt(2))
    psi[2*(i-1)+1] = sparse.csr_matrix((c(i)-cd(i))*(-1j/np.sqrt(2)))

### 2.a. Check that fermions satisfy algebra

$\{\psi_i, \psi_j\} = \delta_{ij}$

In [5]:
psi_algebra_satisfied = True
for i in range(N):
    psi_i = psi[i]

    for j in range(N):
        psi_j = psi[j]

        ac_pi_pj = anticommutator(psi_i, psi_j)

        if i==j:
            if not np.allclose(ac_pi_pj.toarray(), np.identity(N_DIM)):
                psi_algebra_satisfied = False
                
        else:
            if not np.allclose(ac_pi_pj.toarray(), np.zeros((N_DIM, N_DIM))):
                psi_algebra_satisfied = False
        
print(f"Fermion algebra satisfied: {psi_algebra_satisfied}")

Fermion algebra satisfied: True


## 3. Define different Hamiltonian-creation functions

### 3. a. Pre-compute pairwise inner-products of fermions

Saves us some time and compute

In [6]:
# Compute pairwise inner products
psi_pairs = [None for i in range(N**2)] #np.zeros((N, N, N_DIM, N_DIM), dtype=np.complex128)

tic = time.time()
for i in range(N):
    for j in range(i+1, N):
        index = i*N+j
        psi_pairs[index] = psi[i]@psi[j]
toc = time.time()
duration = toc - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

Duration: 0.0 minutes, 0.01598834991455078 seconds


### 3.b Define different Hamiltonian-creation functions to be tested

In [12]:
def H4_base(js): #js being the random coefficients

    # Compute Hamiltonian
    H = sparse.csr_matrix(np.zeros((N_DIM, N_DIM), dtype=np.complex128))
    for i in range(N-3):
        for j in range(i+1, N-2):
            for k in range(j+1, N-1):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*(psi_pairs[i*N+j]@psi_pairs[k*N+l])

    return H

@jit(nopython=True)
def H4_njit(js): #js being the random coefficients

    # Compute Hamiltonian|
    H = sparse.csr_matrix(np.zeros((N_DIM, N_DIM), dtype=np.complex128))
    for i in range(N-3):
        for j in range(i+1, N-2):
            for k in range(j+1, N-1):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*(psi_pairs[i*N+j]@psi_pairs[k*N+l])

    return H

@jit(nopython=True, parallel=True)
def H4_njit_parallel(js): #js being the random coefficients

    # Compute Hamiltonian
    H = sparse.csr_matrix(np.zeros((N_DIM, N_DIM), dtype=np.complex128))
    for i in range(N-3):
        for j in range(i+1, N-2):
            for k in range(j+1, N-1):
                for l in range(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*(psi_pairs[i*N+j]@psi_pairs[k*N+l])

    return H

@jit(nopython=False)
def H4_njit_parallel_prange(js): #js being the random coefficients

    # Compute Hamiltonian
    H = scipy.sparse.csr_matrix(np.zeros((N_DIM, N_DIM), dtype=np.complex128))
    for i in prange(N-3):
        for j in prange(i+1, N-2):
            for k in prange(j+1, N-1):
                for l in prange(k+1, N):
                    H += (1j**(Q/2))*js[i, j, k, l]*(psi_pairs[i*N+j]@psi_pairs[k*N+l])

    return H

In [32]:
sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_all = [np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)])) for j in range(N_SAMPLES+1)]

H4_base_test = H4_base(js_test).toarray()
print(linalg.ishermitian(H4_base_test))

True


## 4. Run tests

In [ ]:
sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_all = [np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)])) for j in range(N_SAMPLES+1)]

# Testing/timing function
def test_H4(H4_func, js_all):
    assert(len(js_all)==N_SAMPLES+1)

    # Execute on N_SAMPLES in parallel
    tic = time.time()
    Parallel(n_jobs=N_JOBS)(delayed(H4_func)(js_all[i]) for i in range(1, N_SAMPLES+1))
    duration = time.time() - tic
    return duration

# Instantiate functions so that they're compiled by cuda
just_run_this_cell_multiple_times = """
tic = time.time()
test_H4(H4_base, js_all)
test_H4(H4_njit, js_all)
test_H4(H4_njit_parallel, js_all)
test_H4(H4_njit_parallel_prange, js_all)
duration = time.time() - tic
print("Kernels instantiated: {duration//60} minutes, {duration%60} seconds")"""


functions_to_test = {"H4_base": {"function": H4_base}}
tic = time.time()

for fname, fdict in functions_to_test.items():
    print(f"\n{fname}:")
    func = fdict["function"]
    duration_i = test_H4(func, js_all)
    fdict["duration"] = duration_i
    print(f"  Total duration: {duration_i//60} minutes, {duration_i%60} seconds")
    print(f"  Average time per Hamiltonian: {duration_i/N_SAMPLES//60} minutes, {duration_i/N_SAMPLES%60} seconds")

duration = time.time() - tic
print(f"\n\nTesting all functions: {duration//60} minutes, {duration%60} seconds")


## 5. Check that all the functions are logically the same (they should be)

In [ ]:
np.random.seed(1)
js_test = np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)]))

H4_base_test = H4_base(js_test).toarray()
H4_njit_test = H4_njit(js_test).toarray()
H4_njit_parallel_test = H4_njit_parallel(js_test).toarray()
H4_njit_parallel_prange_test = H4_njit_parallel_prange(js_test).toarray()

print(np.allclose(H4_base_test, H4_njit_test))
print(np.allclose(H4_njit_parallel_test, H4_njit_parallel_prange_test))
print(np.allclose(H4_base_test, H4_njit_parallel_prange_test))